In [1]:
import sys, os, glob, time
sys.path.append("../src")
os.environ["MPLBACKEND"] = "Agg"

start_time = time.time()

import cv2
from tophat_optimization_l import tophat_extract_l_channel
from contour_crop import contour_crop_eye
from homomorphic_filter import homomorphic_filter_color

RAW_DIR = os.path.abspath("../data/raw_images")
OUT_DIR = os.path.abspath("../data/processed_images")
SUFFIX  = "_processed_pipelinetest9"

os.makedirs(OUT_DIR, exist_ok=True)

# gather images
exts = ("*.jpg","*.JPG","*.jpeg","*.JPEG","*.png","*.PNG","*.tif","*.tiff","*.bmp")
files = [p for e in exts for p in glob.glob(os.path.join(RAW_DIR, e))]
files.sort()

if not files:
    print(f"No images found in {RAW_DIR}")
    raise SystemExit(0)

ok = fail = 0
for in_path in files:
    try:
        # 1) Top-hat optimization (L channel)
        top_hat_img, fname = tophat_extract_l_channel(in_path)

        # 2) Homomorphic filtering in the color space
        hom_fil_img, fname = homomorphic_filter_color(top_hat_img, fname)

        # 3) Contour crop eye from processed image
        cropped_img, fname = contour_crop_eye(hom_fil_img, fname)

        # 4) Save with same name + suffix
        base, ext = os.path.splitext(fname)
        out_name = f"{base}{SUFFIX}{ext or '.jpg'}"
        out_path = os.path.join(OUT_DIR, out_name)
        if not cv2.imwrite(out_path, cropped_img):
            raise RuntimeError("cv2.imwrite returned False")
        ok += 1
        print(f"[OK] {os.path.basename(in_path)} -> {out_name}")
    except Exception as e:
        fail += 1
        print(f"[FAIL] {os.path.basename(in_path)}: {e}")

print(f"\nDone. Saved {ok}. Failed {fail}. Output: {OUT_DIR}")

elapsed = time.time() - start_time
print(f"\nTotal processing time: {elapsed: 2f} seconds")



[OK] T0004-04-06-2019_BL (2).JPG -> T0004-04-06-2019_BL (2)_processed_pipelinetest9.JPG
[OK] T0011-06-06-2019_N.JPG -> T0011-06-06-2019_N_processed_pipelinetest9.JPG
[OK] T0013-06-06-2019_BL (1).JPG -> T0013-06-06-2019_BL (1)_processed_pipelinetest9.JPG
[OK] T0013-06-06-2019_N.JPG -> T0013-06-06-2019_N_processed_pipelinetest9.JPG
[OK] T0017_10-06-2019 (1).JPG -> T0017_10-06-2019 (1)_processed_pipelinetest9.JPG
[OK] T0019_2019-06-10_BL (1).JPG -> T0019_2019-06-10_BL (1)_processed_pipelinetest9.JPG
[OK] T0026_13-06-2019_N.JPG -> T0026_13-06-2019_N_processed_pipelinetest9.JPG
[OK] T0027_13-06-2019_N.JPG -> T0027_13-06-2019_N_processed_pipelinetest9.JPG
[OK] T0028_13-06-2019_BL (2).JPG -> T0028_13-06-2019_BL (2)_processed_pipelinetest9.JPG
[OK] T0036_17-06-2019_N.JPG -> T0036_17-06-2019_N_processed_pipelinetest9.JPG
[OK] T0040_18-06-2019_N.JPG -> T0040_18-06-2019_N_processed_pipelinetest9.JPG
[OK] T0043_19-06-2019_N.JPG -> T0043_19-06-2019_N_processed_pipelinetest9.JPG
[OK] T0044_19-06-201

KeyboardInterrupt: 